In [2]:
pip install nba-api

  Using cached nba_api-1.1.8-py3-none-any.whl (217 kB)
Note: you may need to restart the kernel to use updated packages.


In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import commonteamroster, leaguestandings, teamyearbyyearstats,\
teamdashboardbyyearoveryear,playercareerstats


## get all teams and their season stats

In [81]:
all_teams = teams.get_teams()
team_stats = []
for team in all_teams:
    team_stats.append(teamyearbyyearstats.TeamYearByYearStats(team_id=team["id"]).get_data_frames()[0])

teams_df = pd.concat(team_stats)
teams_df.head()
teams_df_copy = teams_df.copy()

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [87]:
# Remove some unneeded columns
drop_columns = ["CONF_RANK", "DIV_RANK","WINS", "LOSSES", "CONF_COUNT", "DIV_COUNT"]
teams_df.drop(drop_columns, axis=1, inplace=True, errors="ignore")
print(teams_df.columns)
teams_df.head()


Index(['TEAM_ID', 'TEAM_CITY', 'TEAM_NAME', 'YEAR', 'GP', 'WIN_PCT', 'PO_WINS',
       'PO_LOSSES', 'NBA_FINALS_APPEARANCE', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'PF', 'STL', 'TOV', 'BLK', 'PTS', 'PTS_RANK'],
      dtype='object')


,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WIN_PCT,PO_WINS,PO_LOSSES,NBA_FINALS_APPEARANCE,FGM,...,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
0,1610612737,Tri-Cities,Blackhawks,1949-50,64,0.453,1,2,N/A,1818,...,0,0,0,1330,2057,0,0,0,5313,10
1,1610612737,Tri-Cities,Blackhawks,1950-51,68,0.368,0,0,N/A,1988,...,0,0,0,1476,2092,0,0,0,5730,3
2,1610612737,Milwaukee,Hawks,1951-52,66,0.258,0,0,N/A,1674,...,0,0,0,1229,1848,0,0,0,4833,10
3,1610612737,Milwaukee,Hawks,1952-53,71,0.380,0,0,N/A,1873,...,0,0,0,1427,2120,0,0,0,5389,9
4,1610612737,Milwaukee,Hawks,1953-54,72,0.292,0,0,N/A,1757,...,0,0,0,1298,1771,0,0,0,5038,9


In [98]:
# First time three point line
print(teams_df[teams_df["FG3A"]==0]["YEAR"].max())
#First time further statistics as REB, TOV
print(teams_df[teams_df["REB"]==0]["YEAR"].max())

1981-82
1972-73


In [213]:
# How many entries would fall if only consider after 1981-82 season
new_teams_df = teams_df[teams_df["YEAR"]>"1982-83"].copy()
print("Rows before/after: {}/{}".format(len(teams_df), len(new_df)))

# Fill nba finals appearance with 0 where N/A
new_teams_df.loc[new_teams_df["NBA_FINALS_APPEARANCE"]=="N/A","NBA_FINALS_APPEARANCE"] = 0
new_teams_df

Rows before/after: 1507/1066


,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WIN_PCT,PO_WINS,PO_LOSSES,NBA_FINALS_APPEARANCE,FGM,...,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
34,1610612737,Atlanta,Hawks,1983-84,82,0.488,2,3,0,3230,...,1112,2232,3344,1827,2091,626,1329,558,8321,23
35,1610612737,Atlanta,Hawks,1984-85,82,0.415,0,0,0,3444,...,1161,2345,3506,2009,2047,665,1475,541,8743,20
36,1610612737,Atlanta,Hawks,1985-86,82,0.610,4,5,0,3447,...,1249,2405,3654,2025,2170,736,1483,434,8906,17
37,1610612737,Atlanta,Hawks,1986-87,82,0.695,4,5,0,3435,...,1350,2478,3828,2077,2152,700,1279,511,9024,12
38,1610612737,Atlanta,Hawks,1987-88,82,0.610,6,6,0,3443,...,1228,2379,3607,2062,2050,635,1225,537,8844,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,1610612766,Charlotte,Hornets,2015-16,82,0.585,3,4,0,3036,...,734,2869,3603,1778,1487,595,1029,438,8479,11
26,1610612766,Charlotte,Hornets,2016-17,82,0.439,0,0,0,3093,...,721,2853,3574,1891,1360,571,942,390,8601,16
27,1610612766,Charlotte,Hornets,2017-18,82,0.439,0,0,0,3197,...,827,2901,3728,1770,1409,559,1041,373,8874,10
28,1610612766,Charlotte,Hornets,2018-19,82,0.476,0,0,0,3297,...,814,2778,3592,1905,1550,591,1001,405,9081,19


In [218]:
# Create a new dataframe where the WIN_PCT and YEAR are togehter with the stats from the previous year
keep_columns = ["TEAM_ID","TEAM_CITY", "TEAM_NAME","YEAR","WIN_PCT", "GP"]
before_season_df = new_teams_df[keep_columns].copy()
before_season_df["PREV_YEAR"] = before_season_df["YEAR"].apply(lambda x: str(int(x[0:4])+-1).zfill(2)+"-"+str(int(x[5:7])-1).zfill(2))
before_season_df = pd.merge(before_season_df,new_teams_df, left_on=["PREV_YEAR", "TEAM_ID"], \
                            right_on=["YEAR","TEAM_ID"], how="left")
drop_columns= [x+"_y" for x in keep_columns if x!="TEAM_ID"]
before_season_df.drop(drop_columns, axis=1, inplace=True)
before_season_df.rename(columns={"TEAM_CITY_x":"TEAM_CITY", "TEAM_NAME_x":"TEAM_NAME","YEAR_x":"YEAR",\
                                 "WIN_PCT_x":"WIN_PCT","GP_x":"GP"}, inplace=True)
before_season_df = before_season_df[before_season_df["YEAR"]>"1983-84"]
print(before_season_df.isna().sum())
before_season_df[before_season_df.isna().any(axis=1)]

TEAM_ID                   0
TEAM_CITY                 0
TEAM_NAME                 0
YEAR                      0
WIN_PCT                   0
GP                        0
PREV_YEAR                 0
PO_WINS                  37
PO_LOSSES                37
NBA_FINALS_APPEARANCE    37
FGM                      37
FGA                      37
FG_PCT                   37
FG3M                     37
FG3A                     37
FG3_PCT                  37
FTM                      37
FTA                      37
FT_PCT                   37
OREB                     37
DREB                     37
REB                      37
AST                      37
PF                       37
STL                      37
TOV                      37
BLK                      37
PTS                      37
PTS_RANK                 37
dtype: int64


,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,WIN_PCT,GP,PREV_YEAR,PO_WINS,PO_LOSSES,NBA_FINALS_APPEARANCE,...,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
16,1610612737,Atlanta,Hawks,1999-00,0.341,82,1998--1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,1610612738,Boston,Celtics,1999-00,0.427,82,1998--1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,1610612739,Cleveland,Cavaliers,1999-00,0.390,82,1998--1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,1610612740,New Orleans,Hornets,2002-03,0.573,82,2001-02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,1610612741,Chicago,Bulls,1999-00,0.207,82,1998--1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,1610612742,Dallas,Mavericks,1999-00,0.488,82,1998--1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219,1610612743,Denver,Nuggets,1999-00,0.427,82,1998--1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,1610612744,Golden State,Warriors,1999-00,0.232,82,1998--1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293,1610612745,Houston,Rockets,1999-00,0.415,82,1998--1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330,1610612746,Los Angeles,Clippers,1999-00,0.183,82,1998--1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
team_stats = teamyearbyyearstats.TeamYearByYearStats(team_id="1610612737").get_data_frames()
team_stats

[       TEAM_ID   TEAM_CITY   TEAM_NAME     YEAR  GP  WINS  LOSSES  WIN_PCT  \
 0   1610612737  Tri-Cities  Blackhawks  1949-50  64    29      35    0.453   
 1   1610612737  Tri-Cities  Blackhawks  1950-51  68    25      43    0.368   
 2   1610612737   Milwaukee       Hawks  1951-52  66    17      49    0.258   
 3   1610612737   Milwaukee       Hawks  1952-53  71    27      44    0.380   
 4   1610612737   Milwaukee       Hawks  1953-54  72    21      51    0.292   
 ..         ...         ...         ...      ...  ..   ...     ...      ...   
 66  1610612737     Atlanta       Hawks  2015-16  82    48      34    0.585   
 67  1610612737     Atlanta       Hawks  2016-17  82    43      39    0.524   
 68  1610612737     Atlanta       Hawks  2017-18  82    24      58    0.293   
 69  1610612737     Atlanta       Hawks  2018-19  82    29      53    0.354   
 70  1610612737     Atlanta       Hawks  2019-20  67    20      47    0.299   
 
     CONF_RANK  DIV_RANK  ...  OREB  DREB   REB   

In [45]:
teamdashboard = teamdashboardbyyearoveryear.TeamDashboardByYearOverYear(team_id="1610612737").get_data_frames()
teamdashboard

[  GROUP_SET GROUP_VALUE  GP   W   L  W_PCT     MIN   FGM   FGA  FG_PCT  ...  \
 0   Overall     2019-20  67  20  47  0.299  3256.0  2723  6067   0.449  ...   
 
    TOV_RANK  STL_RANK  BLK_RANK  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  \
 0         1         1         1          1        1         1         1   
 
    PLUS_MINUS_RANK  CFID  CFPARAMS  
 0                1   211   2019-20  
 
 [1 rows x 56 columns],
    GROUP_SET GROUP_VALUE  GP   W   L  W_PCT          MIN   FGM   FGA  FG_PCT  \
 0    By Year     2019-20  67  20  47  0.299  3256.000000  2723  6067   0.449   
 1    By Year     2018-19  82  29  53  0.354  3971.000000  3392  7524   0.451   
 2    By Year     2017-18  82  24  58  0.293  3941.000000  3130  7015   0.446   
 3    By Year     2016-17  82  43  39  0.524  3976.000000  3123  6918   0.451   
 4    By Year     2015-16  82  48  34  0.585  3966.000000  3168  6923   0.458   
 5    By Year     2014-15  82  60  22  0.732  3946.000000  3121  6699   0.466   
 6    By Year  

In [106]:
player_stats = playercareerstats.PlayerCareerStats(player_id="201952").get_data_frames()
player_stats_post = player_stats[2]
player_stats_post.head()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,201952,2009-10,00,1610612737,ATL,22.0,9,0,59.0,7,...,0.000,0,2,2,4,3,1,4,3,16
1,201952,2010-11,00,1610612737,ATL,23.0,8,6,238.0,37,...,0.826,0,17,17,28,6,3,8,10,94
2,201952,2011-12,00,1610612737,ATL,24.0,6,6,225.0,30,...,0.895,4,18,22,25,5,5,10,18,84
3,201952,2012-13,00,1610612737,ATL,25.0,6,6,213.0,27,...,0.821,3,14,17,30,9,2,14,17,80
4,201952,2013-14,00,1610612737,ATL,26.0,7,7,242.0,44,...,0.950,5,21,26,35,7,4,21,15,135


## New heading

# New heading

# New heading

# New heading